In [74]:
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [75]:
with open('cam_data.json', 'r') as file:
    cam_data = json.load(file)

    

In [76]:
scaler = MinMaxScaler()
cam0 = np.array(list(map(lambda x: scaler.fit_transform(x['0']), cam_data)))
cam1 = np.array(list(map(lambda x: scaler.fit_transform(x['1']), cam_data)))

In [77]:
cam0_avg = cam0.mean(axis=0).flatten().tolist()
cam0_max = cam0.max(axis=0).flatten().tolist()
cam1_avg = cam1.mean(axis=0).flatten().tolist()
cam1_max = cam1.max(axis=0).flatten().tolist()

In [78]:
with open('cam_agregated.json', 'w') as file:
    json.dump([
        {'avg': cam0_avg, 'max': cam0_max},
        {'avg': cam1_avg, 'max': cam1_max},
    ], file)

### Run predictions

In [3]:
from keras.models import load_model

In [5]:
model = load_model('models/model_2.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                62730     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total para

In [7]:
%run utils.py

In [8]:
image_size = 28
frame_duration = 1.0
overlap = 0.5
X, y = make_data_pipeline(file_names,labels,image_size,frame_duration,overlap)

Generating training data...
Interpolating 117/117nterpolating 53/117Interpolating 99/117Interpolating 1/112Interpolating 52/112Interpolating 100/112Interpolating 1/111Interpolating 49/111Interpolating 94/111Interpolating 1/109Interpolating 50/109Interpolating 91/109Interpolating 1/113Interpolating 63/113Interpolating 107/113Interpolating 1/104Interpolating 54/104Interpolating 99/104Interpolating 1/101Interpolating 51/101Interpolating 95/101Interpolating 1/102Interpolating 53/102Interpolating 96/102Interpolating 1/107Interpolating 53/107Interpolating 99/107Interpolating 1/113Interpolating 53/113Interpolating 99/113Interpolating 1/115Interpolating 53/115Interpolating 100/115Interpolating 1/139Interpolating 52/139Interpolating 98/139Interpolating 1/123Interpolating 42/123Interpolating 81/123Interpolating 102/123Interpolating 117/123Interpolating 1/118Interpolating 23/118Interpolating 44/118Interpolating 54/118Interpolating 81/118Interpolating 101/118Interpolating 115/118Interpolating 1/12